# Differential regulation (ANCOVA for two groups)

This API example shows the functionality in the
[`acore.differential_regulation`](acore.differential_regulation) module:

- Analysis of Covariance (ANCOVA) for two groups

Then we can do the same for examples with three
and more groups, where a omnibus analysis across groups
is combined with posthoc anlysis for separate groups.

The functions are the same for both cases. The `group1` and
`group2` columns give the posthoc comparison.

Using vuecore we add:
- [ ] include a PCA colored by groups as well as covariance factors

In [ ]:
%pip install acore

In [ ]:
import dsp_pandas
import pandas as pd

import acore.differential_regulation as ad

dsp_pandas.format.set_pandas_options(
    max_columns=9,
    max_colwidth=20,
)

In [ ]:
BASE = (
    "https://raw.githubusercontent.com/RasmussenLab/njab/"
    "HEAD/docs/tutorial/data/alzheimer/"
)
CLINIC_ML: str = "clinic_ml.csv"  # clinical data
OMICS: str = "proteome.csv"  # omics data
freq_cutoff: float = (
    0.7  # at least x percent of samples must have a value for a feature (here: protein group)
)
#
covariates: list[str] = ["age", "male"]
group: str = "AD"
subject_col: str = "Sample ID"
factor_and_covars: list[str] = [group, *covariates]

# BASE = (
#     "https://raw.githubusercontent.com/Multiomics-Analytics-Group/acore/"
#     "HEAD/example_data/MTBLS13311/"
#     ""
# )
# CLINIC_ML: str = "MTBLS13411_meta_data.csv"  # clinical data
# OMICS: str = "MTBLS13411_processed_data.csv"  # omics data
# covariates: list[str] = []
# group: str = "Factor Value[Strain type]"
# subject_col: str | int = 0
# factor_and_covars: list[str] = [group, *covariates]

# ANCOVA analysis for two groups
Use combined dataset for ANCOVA analysis.

In [ ]:
omics_and_clinic = pd.read_csv(
    "../../example_data/alzheimer_proteomics/alzheimer_example_omics_and_clinic.csv",
    index_col=subject_col,
)
omics_and_clinic

metadata here is of type integer. All floats are proteomics measurements.

In [ ]:
omics_and_clinic.dtypes.value_counts()

In [ ]:
omics_and_clinic[[group, *covariates]]

run ANCOVA analysis

In [ ]:
# omics_and_clinic = omics_and_clinic.astype(float)
# ? this is no needed for run_ancova (the regex where groups are joined)
ancova = (
    ad.run_ancova(
        omics_and_clinic.astype({group: str}),  # ! target needs to be of type str
        # subject=subject_col, # not used
        drop_cols=[],
        group=group,  # needs to be a string
        covariates=covariates,
    )
    .set_index("identifier")
    .sort_values(by="posthoc padj")
)  # need to be floats?
ancova_acore = ancova
ancova

The first columns contain group averages for each group for the specific
protein group

In [ ]:
ancova.iloc[:, :6]

The others contain the test results (based on a linear model) for each protein group
(on each row). Some information is duplicated.

In [ ]:
regex_filter = "pval|padj|reject|post"
ancova.filter(regex=regex_filter)

The other information is about fold-changes and other information.

In [ ]:
ancova.iloc[:, 6:].filter(regex=f"^(?!.*({regex_filter})).*$")